### Get all info for each cell in scRNA-seq data

Not in used as we do not need SRA detail

In [1]:
import pandas as pd

In [2]:
cluster_info_df = pd.read_csv('../data/HN_patient_specific/AS_et_al_cellInfo2.csv', index_col=0)

sra_run_df = pd.read_csv('../data/HN_patient_specific/AS_et_at_SraRunTable.txt', index_col='Run')

sra_summary_df = pd.read_csv('../data/HN_patient_specific/AS_et_al_sra_result.csv', index_col='Sample Accession')
# Drop Instrument as it is already in sra_run_df
sra_summary_df = sra_summary_df.drop('Instrument', axis=1)

FileNotFoundError: [Errno 2] File b'../data/HN_patient_specific/AS_et_al_cellInfo2.csv' does not exist: b'../data/HN_patient_specific/AS_et_al_cellInfo2.csv'

In [3]:
print (cluster_info_df.shape, len(set(cluster_info_df.index)))
cluster_info_df.head()

NameError: name 'cluster_info_df' is not defined

##### Run file

In [4]:
print (sra_run_df.shape, len(set(sra_run_df.index)))
# sra_run_df.head(1).T
sra_run_df[sra_run_df['GEO_Accession'].isin(['GSM3313660'])].T

NameError: name 'sra_run_df' is not defined

Drop smaller library size

In [87]:
sra_run_df = sra_run_df.sort_values(by=['GEO_Accession', 'MBases'], ascending=[True, False])

In [91]:
sra_run_df[sra_run_df['GEO_Accession']=='GSM3313660'][['GEO_Accession', 'MBases']]

,GEO_Accession,MBases
Run,,
SRR7622081,GSM3313660,727
SRR7622080,GSM3313660,237


In [100]:
sra_run_df = sra_run_df.drop_duplicates(subset=['GEO_Accession'], keep='first')
sra_run_df = sra_run_df.reset_index().set_index('sample_acc')

##### Summary file

In [101]:
sra_summary_df.loc[:, 'Cell ID'] = [c.split(' ')[1] for c in sra_summary_df['Experiment Title']]
sra_summary_df.loc[:, 'GEO_Accession'] = [c.split(': ')[0] for c in sra_summary_df['Experiment Title']]

In [102]:
print (sra_summary_df.shape, len(set(sra_summary_df.index)))
sra_summary_df.head(1).T

(1431, 17) 1431


Sample Accession,SRS3611000
Experiment Accession,SRX4486839
Experiment Title,GSM3314329: RHC6653 (scRNA-seq); Homo sapiens;...
Organism Name,Homo sapiens
Submitter,GEO
Study Accession,SRP155751
Study Title,"Single cell RNA-seq profiles of primary, metas..."
Sample Title,NaN
"Total Size, Mb",1.42
Total RUNs,1
Total Spots,32549


In [103]:
sra_df = pd.merge(sra_run_df, sra_summary_df, left_index=True, right_index=True)
sra_df = sra_df.reset_index()
sra_df.columns = ['Sample Accession'] + list(sra_df.columns[1:])
sra_df = sra_df.set_index('Cell ID')

In [104]:
print (len(set(sra_df.index)), len(sra_df.index))

1431 1431


In [105]:
cell_meta_df = pd.merge(sra_df, cluster_info_df, how='left', left_index=True, right_index=True)

In [109]:
cell_meta_df.to_csv('../data/HN_patient_specific/AS_et_al_cell_metadata.csv')